# SAT formulation for Multiple Couriers planning

## 1. Direct constraint modelling

#### **Testing**: retrieve the first feasible solution

In [103]:
from z3 import *
import numpy as np
#from itertools import combinations
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    # 1) Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2) Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3) Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4) Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    # 5) Each item is taken exactly once among all couriers and across all timesteps
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    # 6) All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    '''# 7) Symmetry breaking: two couriers have the same load size
    for c1, c2 in combinations(range(m), 2):
        if l[c1] == l[c2]:
            for t in range(1, max_times):
                for i in range(n):
                    solver.add(Or(Not(v[c1][i][t]), v[c2][i][t]))'''
    
    # return model and the updated 3D variable
    return solver, v

# Calculate distances relatively to the paths retrieved as solution
def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Print the total distance for each courier
def print_total_distances(solution, distances_dict):    
    for courier in distances_dict.keys():
        print(f"Courier {courier + 1} path: {solution[courier]}")
        print(f"Courier {courier + 1} total distance: {distances_dict[courier]}")

# Retrieve the solution
def extract_solution(model, instance, v):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times+1):
            for i in range(n+1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Importing instance
instance_num = "05"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

# Running the model
solver, v = mcp(instance=instance)

# Check satisfiability
if solver.check() == sat: # type: ignore
    model = solver.model()
    solution = extract_solution(model, instance, v)
    distances_dict = compute_distances(solution, instance)
    print_total_distances(solution, distances_dict)
    max_dist = compute_max_dist(distances_dict)
    print(f"\nObj. function value, max dist: {max_dist}")
else: print("unsat")

Courier 1 path: [4, 2, 4]
Courier 1 total distance: 160
Courier 2 path: [4, 1, 3, 4]
Courier 2 total distance: 206

Obj. function value, max dist: 206


#### trying to optimize distance constraint:
- pro: it returns unsat for unfeasible upperbounds (e.g. upperbound = 1)
- cons: it uses pseudobooleans; it returns only the first feasible solution -> no optimization process

In [59]:
from z3 import *
import numpy as np
#from itertools import combinations
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    # 1) Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2) Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3) Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4) Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    # 5) Each item is taken exactly once among all couriers and across all timesteps
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    # 6) All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    # 7) Symmetry breaking: two couriers have the same load size
    for c1, c2 in combinations(range(m), 2):
        if l[c1] == l[c2]:
            for t in range(1, max_times):
                for i in range(n):
                    solver.add(Or(Not(v[c1][i][t]), v[c2][i][t]))
    
    # return model and the updated 3D variable
    return solver, v

# 6. Distance computation
def distance_constraint(instance, solver, v, upper_bound):
    m = instance["m"]
    n = instance["n"]
    max_times = instance["max_times"]
    D = instance["D"]

    for c in range(m):
        total_distance = Int(f"total_distance_{c}")
        distance_sum = []

        for t in range(1, max_times-1):  # Only considers no-depot steps
            for i in range(n):
                for j in range(n):
                    if i != j:
                        # Sum distance only if the courier moves from i to j between consecutive timesteps
                        distance_sum.append(If(And(v[c][i][t], v[c][j][t + 1]), D[i][j], 0))

            # Add depot to first item and last item back to depot distances
            distance_sum.append(If(v[c][n][0] == True, D[n][i], 0))  # Depot to first item

        solver.add(total_distance == Sum(distance_sum))
        solver.add(total_distance <= upper_bound)

# Calculate distances relatively to the paths retrieved as solution
def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Print the total distance for each courier
def print_total_distances(solution, distances_dict):    
    for courier in distances_dict.keys():
        print(f"Courier {courier + 1} path: {solution[courier]}")
        print(f"Courier {courier + 1} total distance: {distances_dict[courier]}")

# Retrieve the solution
def extract_solution(model, instance, v):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Importing instance
instance_num = "05"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

# Running the model
solver, v = mcp(instance=instance)

# Add distance constraint
#distance_constraint(instance, instance["max_dist"], v)
distance_constraint(instance, solver, v, 500)

# Check satisfiability
if solver.check() == sat: # type: ignore
    model = solver.model()
    solution = extract_solution(model, instance, v)
    distances_dict = compute_distances(solution, instance)
    print_total_distances(solution, distances_dict)
    max_dist = compute_max_dist(distances_dict)
    print(f"\nObj. function value, max dist: {max_dist}")
else: print("unsat")

Courier 1 path: [4, 2, 4]
Courier 1 total distance: 160
Courier 2 path: [4, 1, 3, 4]
Courier 2 total distance: 206

Obj. function value, max dist: 206


In [163]:
from z3 import *
import os

# Utility function to encode constraints
from encodings_utils import *

def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Maximum load each courier can carry
    s = instance["s"]  # Size of each item
    D = instance["D"]  # Distance matrix
    max_times = instance["max_times"]  # Maximum number of time steps

    solver = Solver()
    solver.set("timeout", timeout)

    # Variables: x_{c}_{i}_{t} indicating if courier c picks item i at time t
    x = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]

    # Constraints for item assignment: Each item must be picked exactly once
    for i in range(1, n + 1):
        solver.add(exactly_one_seq([x[c][i-1][t] for c in range(m) for t in range(1, max_times + 1)], f"assign_item_{i}"))

    # Constraints for couriers: A courier can pick at most one item at a time
    for c in range(m):
        for t in range(1, max_times + 1):
            solver.add(at_most_one_seq([x[c][i-1][t] for i in range(1, n + 1)], f"courier_{c}_time_{t}"))

    # Constraints for load capacity
    for c in range(m):
        load = sum(s[i-1] * sum(x[c][i-1][t] for t in range(1, max_times + 1)) for i in range(1, n + 1))
        solver.add(load <= l[c])

    # Calculate travel time for each courier
    travel_times = []
    for c in range(m):
        travel_time = 0
        for t in range(1, max_times):
            for i in range(1, n + 1):
                for j in range(1, n + 1):
                    travel_time += If(And(x[c][i-1][t], x[c][j-1][t+1]), D[i][j], 0)
        travel_times.append(travel_time)

    # Iterative deepening to minimize the maximum travel time
    def find_minimum_max_travel_time(lower_bound, upper_bound):
        while lower_bound < upper_bound:
            mid = (lower_bound + upper_bound) // 2

            # Add constraint that the maximum travel time should be <= mid
            solver.push()  # Save current state
            solver.add(And([time <= mid for time in travel_times]))

            if solver.check() == sat:
                upper_bound = mid  # Try to find a smaller maximum
            else:
                lower_bound = mid + 1  # Increase the lower bound

            solver.pop()  # Restore state

        return lower_bound

    # Setting initial bounds
    lower_bound = 0
    upper_bound = sum(max(D[i][j] for i in range(1, n + 1) for j in range(1, n + 1)) for _ in range(m))
    upperbound = 5

    # Find the minimum possible maximum travel time
    min_max_travel_time = find_minimum_max_travel_time(lower_bound, upper_bound)

    # Add the final constraint for the minimum maximum travel time
    solver.add(And([time <= min_max_travel_time for time in travel_times]))

    return solver, x

# Calculate distances relative to the paths retrieved as a solution
def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Retrieve the solution
def extract_solution(model, instance, v):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times+1):
            for i in range(n+1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Print the total distance for each courier
def print_total_distances(solution, distances_dict):
    for courier in distances_dict.keys():
        print(f"Courier {courier + 1} path: {solution[courier]}")
        print(f"Courier {courier + 1} total distance: {distances_dict[courier]}")

# Importing instance
instance_num = "05"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

solver, v = mcp(instance)

if solver.check() == sat:
    model = solver.model()
    solution = extract_solution(model, instance, v)
    distances_dict = compute_distances(solution, instance)
    print_total_distances(solution, distances_dict)
    objective_value = compute_max_dist(distances_dict)
    print(f"Objective Value (Minimum Maximum Travel Time): {objective_value}")
else:
    print("No solution found within the given bounds.")


Courier 1 path: [2]
Courier 1 total distance: 160
Courier 2 path: [1, 3]
Courier 2 total distance: 206
Objective Value (Minimum Maximum Travel Time): 206


## Linear search

In [13]:
from z3 import * # type: ignore
import numpy as np # type: ignore
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver() # type: ignore
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)] # type: ignore
    
    # CONSTRAINTS
    # 1) Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2) Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3) Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4) Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    # 5) Each item is taken exactly once among all couriers and across all timesteps
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    # 6) All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    # Return model and the updated 3D variable
    return solver, v

# 6. The distance should be lower than a given upperbound
def distance_constraint(instance, solver, v, upperbound):
    m = instance["m"]
    n = instance["n"]
    max_times = instance["max_times"]
    D = instance["D"]

    for c in range(m):
        total_distance = Int(f"total_distance_{c}")
        distance_sum = []

        for t in range(1, max_times-1):  # Only considers no-depot steps
            for i in range(n):
                for j in range(n):
                    if i != j:
                        # Sum distance only if the courier moves from i to j between consecutive timesteps
                        distance_sum.append(If(And(v[c][i][t], v[c][j][t + 1]), D[i][j], 0))

            # Add depot to first item and last item back to depot distances
            distance_sum.append(If(v[c][n][0] == True, D[n][i], 0))  # Depot to first item

        solver.add(total_distance == Sum(distance_sum))
        solver.add(total_distance <= upperbound)

# Calculate distances for each courier
def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n+1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot-1][node-1]
                depot = node
        total_distance += distances[depot-1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

# Objective function
def compute_max_dist(distances_dict):
    return max(distances_dict.values())

# Retrieve the model's solution
def extract_solution(model, instance, v):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 1,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Linear search algo: starting from the bottom, since this is a minimization problem
def run_sat_linear_search(instance):
    
    # Initialize the Z3 solver and variables
    solver, v = mcp(instance)  # mcp returns the solver and variables

    # Set the original upper bound and lower bound for linear search
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    # Step size for the linear search 
    step = 2

    # Linear search for finding a feasible solution
    distance_limit = lower_bound
    best_solution = None
    best_objective_value = float('inf') # init to infinite

    # continue until the upperbound is not crossed
    while distance_limit <= upper_bound:

        # Save the current state of the solver
        solver.push()

        # Add the distance constraint to the solver
        distance_constraint(instance, solver, v, upperbound=distance_limit)

        # Check if the problem is satisfiable
        if solver.check() == sat:  # Solution found
            model = solver.model()
            solution = extract_solution(model, instance, v)
            distances_dict = compute_distances(solution, instance)
            objective_value = compute_max_dist(distances_dict)

            # Print intermediate solution and objective value
            print(f"Found feasible solution with distance limit {distance_limit}:")
            print(f"  Objective Function Value: {objective_value}")
            
            # Update the best solution if the current one is better
            if objective_value < best_objective_value:
                best_solution = solution
                best_objective_value = objective_value

        # Increment the distance limit for the next iteration
        distance_limit += step

        # Restore the solver state for the next iteration
        solver.pop()

    # Return the best solution found and its objective value, or "unsat" if no feasible solution
    if best_solution is not None:
        return best_solution, best_objective_value
    else:
        return "unsat", None

# Importing instance
instance_num = "02"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)

# Compute additional info based on the instance
instance.update(compute_additional_info(instance))

# Run the linear search SAT solver
result, obj_value = run_sat_linear_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")

Found feasible solution with distance limit 133:
  Objective Function Value: 464
Found feasible solution with distance limit 135:
  Objective Function Value: 464
Found feasible solution with distance limit 137:
  Objective Function Value: 464
Found feasible solution with distance limit 139:
  Objective Function Value: 464
Found feasible solution with distance limit 141:
  Objective Function Value: 464
Found feasible solution with distance limit 143:
  Objective Function Value: 464
Found feasible solution with distance limit 145:
  Objective Function Value: 464
Found feasible solution with distance limit 147:
  Objective Function Value: 464
Found feasible solution with distance limit 149:
  Objective Function Value: 464
Found feasible solution with distance limit 151:
  Objective Function Value: 464
Found feasible solution with distance limit 153:
  Objective Function Value: 464
Found feasible solution with distance limit 155:
  Objective Function Value: 464
Found feasible solution with

KeyboardInterrupt: 

## Binary search

In [101]:
from z3 import *
import numpy as np
from encodings_utils import *

# Modelling MCP problem
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]

    # - paths
    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)]
          for start in range(n+1)] for i in range(m)]
    
    # channeling between d and v
    for c in range(m):
        for t in range(max_times):
            for startj in range(n+1):
                for endj in range(n+1):
                    solver.add(
                        Implies(And(v[c][startj][t], v[c][endj][t+1]), d[c][startj][endj])
                    )
    
    # CONSTRAINTS
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    for c in range(m):
        solver.add(v[c][n][0] == True)
        solver.add(v[c][n][max_times] == True)
    
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))

    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))
    
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    return solver, v, d

# 6. The distance should be lower than a given upperbound
def distance_constraint(instance, solver, v, d, upperbound):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    max_times = instance["max_times"] # time
    distances = instance["D"] # distances between packages
    for c in range(m):
        dists = []
        for start in range(n+1):
            for end in range(n+1):
                for _ in range(distances[start][end]):
                    dists.append(d[c][start][end])
        solver.add(at_most_k_seq(dists, upperbound, f"Courier_dist_{c}_{upperbound}"))

def compute_distances(solution, instance):
    distances = instance["D"]
    n = instance["n"]
    distances_dict = {}

    for i, courier_solution in enumerate(solution):
        if not courier_solution:
            continue
        total_distance = 0
        depot = n + 1  # Starting node
        for node in courier_solution:
            if node != depot:
                total_distance += distances[depot - 1][node - 1]
                depot = node
        total_distance += distances[depot - 1][n]  # Return to starting node
        distances_dict[i] = total_distance

    return distances_dict

def compute_max_dist(distances_dict):
    return max(distances_dict.values())

def extract_solution(model, instance, v):
    m = instance["m"]
    n = instance["n"]
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if is_true(model[v[c][i][t]]):  # Properly check the model's interpretation of the variable
                    courier_solution.append(i + 1)
        solution.append(courier_solution)
    return solution

def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    max_times = (n // m) + 3
    min_load = min(instance["s"])
    max_load = max(instance["s"])
    max_distance = np.sum(np.max(D, axis=1))*2

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 1,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

def run_sat_binary_search(instance):
    solver, v, d = mcp(instance)
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    best_solution = None
    best_objective_value = float('inf')

    print(f"Binary search on Instance {instance_num}")

    while lower_bound <= upper_bound:
        distance_limit = (lower_bound + upper_bound) // 2
        print(f"\nTrying distance limit: {distance_limit}")

        solver.push()
        distance_constraint(instance, solver, v, d, upperbound=distance_limit)

        if solver.check() == sat:
            model = solver.model()
            solution = extract_solution(model, instance, v)
            distances_dict = compute_distances(solution, instance)
            objective_value = compute_max_dist(distances_dict)

            # print intermediate solutions
            print("Paths:", solution)
            print("Objective value:", objective_value)
            
            if objective_value < best_objective_value:
                best_solution = solution
                best_objective_value = objective_value

            upper_bound = distance_limit - 1
        else:
            lower_bound = distance_limit + 1

        solver.pop()

    if best_solution is not None:
        return best_solution, best_objective_value
    else:
        return "unsat", None

# Importing instance
instance_num = "05"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(compute_additional_info(instance))

# Run the binary search SAT solver
result, obj_value = run_sat_binary_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("\nBest Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")

Binary search on Instance 05

Trying distance limit: 351
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 175
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 87
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 43
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 21
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 10
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 5
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 2
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Trying distance limit: 1
Paths: [[4, 2, 4], [4, 1, 3, 4]]
Objective value: 206

Best Solution Found:
Courier 1: [4, 2, 4]
Courier 2: [4, 1, 3, 4]
Objective Function Value: 206
